In [ ]:
from boardlaw.sql import *

In [ ]:
noise = query('''select * from noise_scales''')

In [ ]:
from analysis import data
ags = data.load()

In [ ]:
import plotnine as pn
from analysis import plot

noise = query('''select * from noise_scales''')
df = pd.merge(ags, noise, left_on='snap_id', right_on='id').query('test_nodes == 64')

Bb = df.batches*df.batch_size 
Bs = df.batch_size

Gb = df.mean_sq
Gs = df.sq_mean

G2 = 1/(Bb - Bs)*(Bb*Gb - Bs*Gs)
S = 1/(1/Bs - 1/Bb)*(Gs - Gb)

df['low_bias'] = S/G2
df['low_var'] = df.batch_size*df.variance/df.mean_sq

(pn.ggplot(df[df.low_var > 0])
     + pn.geom_point(pn.aes(x='data.ELO*elo', y='low_var', color='factor(boardsize)'), shape='.', size=.25, show_legend=False)
#     + pn.facet_wrap('boardsize', ncol=2, labeller='label_both')
     + plot.IEEE()
     + pn.labs(x='Elo v. perfect play', y='noise scale')
     + pn.scale_y_continuous(trans='log10')
     + pn.theme(figure_size=(4, 4)))

In [ ]:
pn.facet_wrap?

In [ ]:
df.query('boardsize == 6')